In [1]:
import os
import re
import sys
import hashlib
import socket
import requests
import json
import time
from time import time
from operator import itemgetter
from datetime import datetime, timezone
from pyp2p.net import *

from config import Config

## Test Blockchain Endpoints

In [35]:
host = 'http://0.0.0.0:5000'

In [41]:
wallet = requests.post(host+'/wallet/new', json={'password': 'lalalend'}).content
wallet

b'{"ADDRESS":"z01x00a61643807d1f0bfe6dbb609311de768fb41430479b6564ca5f2c4f06"}\n'

In [84]:
login = requests.post(host+'/wallet/login', json={'password': 'lalalend'}).content
login

b'{"ADDRESS":"z01x00a61643807d1f0bfe6dbb609311de768fb41430479b6564ca5f2c4f06","MSG":true}\n'

In [69]:
logout = requests.get(host+'/wallet/logout').content
logout

b'{"MSG":true}\n'

In [73]:
tradeTx = {
    'type': 'trade',
    'sender': json.loads(wallet)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 30000.0,
    'send': 'btc',
    'sendVol': 2.0,
    'get': 'usdt',
    'getVol': 60000.0,
    'comissionAmount':2
}

txReq = requests.post(host+'/transactions/new', json=tradeTx).content
txReq

b'{"msg":2}\n'

In [74]:
ordersPool = requests.get(host+'/getTradeOrders').content
ordersPool

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":60000.0,"price":30000.0,"send":"btc","sendVol":2.0,"sender":"z01x00a61643807d1f0bfe6dbb609311de768fb41430479b6564ca5f2c4f06","symbol":"btc/usdt","timestamp":1629442757.094091,"tradeTxId":"be1d1cf782eecf96109bfc7cb921ce1000182770d12bb1b4c27f72b29f708f97"}]}\n'

In [77]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":4,"message":"New Block Forged","previous_hash":"8c11f901c96c883077d0469bda1e0557a08e268ad298baba6d5166e7752d4899","proof":26333,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"f919fea6dfb54643965e127a08347344","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629442817.91131,"tradeTxId":null}]}\n'

In [79]:
try: 
    Config().COINBASE
except:
    print('Write coinbase into file')

Write coinbase into file
